# Chapter 2

## 代表的なクエリのライフサイクル

### Spark Sessionと依存関係, utility関数の準備

In [ ]:
import io
import struct
from pprint import pprint

import pandas as pd
from avro.datafile import DataFileReader
from avro.io import DatumReader

import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

CATALOG = "spark_catalog"
CATALOG_URL = "http://server:8181/"
WAREHOUSE = "s3://amzn-s3-demo-bucket"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.7.1"

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.warehouse", WAREHOUSE)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", CATALOG)
        .getOrCreate()
)

In [ ]:
def list_objects(bucket_name: str, prefix: str) -> None:
    """
    指定されたバケットとプレフィックスに基づいてオブジェクト一覧を表示
    """
    try:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        if 'Contents' in response:
            for obj in response['Contents']:
                last_modified_str = obj['LastModified'].strftime("%Y-%m-%d %H:%M:%S")
                print(f"Key:          {obj['Key']}")
                print(f"LastModified: {last_modified_str}")
                print(f"Size:         {obj['Size']} bytes")
                print("-" * 40)
        else:
            print("オブジェクトが見つかりませんでした。")
            
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")

## boto3 クライアントの準備

以降の手順でminIO上のファイルを参照するため、boto3のS3クライアントを用意します。  
(手順をJupyter内で完結させるためにS3クライアントで操作しますが、適宜minIOの管理GUI`http://localhost:9001/browser`から参照しても構いません)  
  
＊実際にはS3を使用せず、ローカル上のminIOにアクセスしている点に注意してください。minIOはS3互換のオブジェクトストレージであるため、以下のようにboto3から操作できるのです) 
  
＊コード内にminIOのクレデンシャルをハードコードしていますが、**これを本番環境で真似しないでください。**  
これはローカルのハンズオン環境に接続するための簡易的な措置であり、コード内へのクレデンシャルのハードコードは非常にバッドなプラクティスです。本番向けには最低限環境変数に保持するか、適切なシークレットマネージャを使用してください。

In [ ]:
import boto3
import json

# Minioサーバーの設定
minio_endpoint = 'http://minio:9000'
minio_access_key = 'admin'
minio_secret_key = 'password'
bucket_name = 'amzn-s3-demo-bucket'

# S3クライアントを作成
s3 = boto3.client('s3',
                  endpoint_url=minio_endpoint,
                  aws_access_key_id=minio_access_key,
                  aws_secret_access_key=minio_secret_key,
                  verify=False)

# バケットのリストを取得
response = s3.list_buckets()
print("Buckets:")
for bucket in response['Buckets']:
    print(f"- {bucket['Name']}")

### CREATE TABLE

In [ ]:
%sql spark

In [ ]:
spark.sql("""
DROP TABLE IF EXISTS demo.simple_table PURGE
""")

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS demo.simple_table (
      id BIGINT,
      name STRING,
      score DOUBLE
    )
    LOCATION "s3://amzn-s3-demo-bucket/demo/simple_table"
    TBLPROPERTIES ('table_type'='ICEBERG');

テーブルが作成されると、Iceberg は内部でメタデータファイル（**.metadata.json）を生成し、カタログにそのロケーション情報を登録します。  
まずは、以下の SQL コマンドでテーブルの詳細な情報（スキーマやパーティション情報、プロパティなど）を確認してください。  

In [ ]:
%%sql

DESCRIBE EXTENDED demo.simple_table

`CREATE TABLE`で作成されたメタデータを確認します。MinIOの`amzn-s3-demo-bucket/demo/simple_table/metadata`をリストすると、`**.metadata.json`が作成されているのがわかります。これがメタデータファイルです。

In [ ]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

メタデータファイルの中身を確認します。

In [ ]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

## INSERT

先ほど作成したテーブルにレコードを挿入してみましょう。

In [ ]:
%%sql
    
INSERT INTO demo.simple_table VALUES
    (1, 'Alice', 85.5),
    (2, 'Bob', 90.0),
    (3, 'Charlie', 78.0);

データの挿入が完了すると、Iceberg は新しいスナップショットを作成し、メタデータリスト、マニフェストファイル、データファイルを作成します。その上で、メタデータファイルを更新し、カタログにその情報を登録します。  
まずは、以下の SQL コマンドで挿入したデータが正しくテーブルに反映されていることを確認してください。

In [ ]:
%%sql

SELECT * FROM demo.simple_table;

次に、ストレージ上のファイルを観察してみましょう。`metadata`ディレクトリに新しいメタデータファイル、メタデータリスト、マニフェストファイルが作成されていることが分かります。それぞれを順に見ていきます。

In [ ]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

まずは最新のメタデータファイルを確認します。先ほどテーブルを作成した際に確認したメタデータファイルに比べて、INSERT時に作成されたsnapshotの情報が追加されていることが分かります。

In [ ]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

それでは、スナップショットリストを確認します。

In [ ]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    snap_avro_files = [
        obj for obj in response['Contents']
        if obj['Key'].startswith('demo/simple_table/metadata/snap-') and obj['Key'].endswith('.avro')
    ]
    if snap_avro_files:
        snap_avro_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_snap_key = snap_avro_files[0]['Key']
        print(f"最新の Snapshot AVRO ファイル: {latest_snap_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_snap_key)
        content = resp['Body'].read()

        bytes_stream = io.BytesIO(content)
        reader = DataFileReader(bytes_stream, DatumReader())

        print("AVRO ファイルのレコード:")
        with DataFileReader(bytes_stream, DatumReader()) as reader:
            for record in reader:
                pprint(record)
                print()
    else:
        print("snap-***.avro ファイルが見つかりませんでした。")
else:
    print("オブジェクトが見つかりませんでした。")

続いて、マニフェストファイルを確認します。マニフェストファイルには、データファイルのパスや統計情報が含まれています。

In [ ]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

def interpret_iceberg_bound_bytes(bound_value: bytes, column_id: int):
    """
    Icebergテーブルのマニフェストファイル内のバイナリを解釈する
    (coulumn_id ごとの型が事前に把握できている状況を想定したシンプルな実装)
      - col1 (id=1): BIGINT → 64bit 整数 (リトルエンディアン)
      - col2 (id=2): STRING → UTF-8 デコードを試行
      - col3 (id=3): DOUBLE → 64bit 浮動小数 (リトルエンディアン)
    """
    if column_id == 1:
        return struct.unpack('<q', bound_value)[0]
    elif column_id == 2:
        try:
            return bound_value.decode('utf-8')
        except UnicodeDecodeError:
            return bound_value.hex()
    elif column_id == 3:
        return struct.unpack('<d', bound_value)[0]
    else:
        return bound_value.hex()

def decode_bounds(bound_list: list):
    """
    lower_bounds / upper_bounds のような
    [{'key': <column_id>, 'value': <bytes>}, ...] という構造を
    interpret_iceberg_bound_bytes() でデコードする。
    """
    for item in bound_list:
        if isinstance(item['value'], bytes):
            col_id = item['key']
            item['value'] = interpret_iceberg_bound_bytes(item['value'], col_id)


if 'Contents' in response:
    # snap- を含まない .avro → マニフェストファイルを抽出
    manifest_avro_files = [
        obj for obj in response['Contents']
        if obj['Key'].endswith('.avro') and 'snap-' not in obj['Key']
    ]
    if manifest_avro_files:
        manifest_avro_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_manifest_key = manifest_avro_files[0]['Key']
        print(f"最新のマニフェストファイル: {latest_manifest_key}\n")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_manifest_key)
        content = resp['Body'].read()
        bytes_stream = io.BytesIO(content)

        print("マニフェストファイルのレコード:")
        with DataFileReader(bytes_stream, DatumReader()) as reader:
            for record in reader:
                df = record.get('data_file', {})
                if 'lower_bounds' in df:
                    decode_bounds(df['lower_bounds'])
                if 'upper_bounds' in df:
                    decode_bounds(df['upper_bounds'])

                pprint(record)
                print()
    else:
        print("マニフェストファイル (.avro かつ snap- 以外) が見つかりませんでした。")
else:
    print("オブジェクトが見つかりませんでした。")

最後にデータファイルを観察します。Icebergのデータファイルの形式は Parquet, ORC, Avro から選択可能ですが、今回はデフォルトの Parquet でデータファイルが作成されていることがわかります。

In [ ]:
prefix = 'demo/simple_table/data/'
list_objects(bucket_name, prefix)

## UPDATE

次に、レコードをアップデートしてみましょう。以下の例では、id=2 のレコードの `score` を 100.0 に更新しています。  

In [ ]:
%%sql
    
SELECT * FROM demo.simple_table

In [ ]:
%%sql
    
UPDATE demo.simple_table SET score = 100.0 WHERE id = 2;

In [ ]:
%%sql
    
SELECT * FROM demo.simple_table

ここで改めて、ストレージ上のファイルを観察してみましょう。`metadata` ディレクトリにメタデータファイル、メタデータリスト、マニフェストファイルが作成されていることが分かります。

In [ ]:
prefix = 'demo/simple_table/metadata/'
list_objects(bucket_name, prefix)

In [ ]:
prefix = 'demo/simple_table/metadata/'
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

if 'Contents' in response:
    metadata_files = [
        obj for obj in response['Contents'] 
        if obj['Key'].endswith('.metadata.json')
    ]
    
    if len(metadata_files) == 0:
        print("メタデータファイル(.metadata.json)が見つかりませんでした。")
    else:
        # LastModified が最新のファイルを選択
        metadata_files.sort(key=lambda x: x['LastModified'], reverse=True)
        latest_metadata_key = metadata_files[0]['Key']

        print(f"最新のメタデータファイル: {latest_metadata_key}")

        resp = s3.get_object(Bucket=bucket_name, Key=latest_metadata_key)
        content = resp['Body'].read().decode('utf-8')
        data = json.loads(content)

        print(json.dumps(data, indent=2))
else:
    print("指定したパスにオブジェクトが存在しませんでした。")

データファイルに関しても、更新前と比較して、新しいデータファイルが作成されていることが分かります。

In [ ]:
prefix = 'demo/simple_table/data/'
list_objects(bucket_name, prefix)

このように、Iceberg テーブルの更新時には新しいメタデータファイル、マニフェストリスト、マニフェストファイル、データファイルが作成されます。更新前のファイルはそのまま残り、メタデータファイル→マニフェストリスト→マニフェストファイル→データファイルのツリー構造のメタデータが更新されることで、データの変更を管理しています。

更新前のファイルが残っているので、過去のスナップショット ID を指定することで、過去のデータを参照することが可能です。これがタイムトラベルクエリです。  
スナップショット ID はメタデータファイルなどからも確認できますが、Icebergにはテーブルのスナップショットのログの参照に利用できる便利なメタデータテーブルが用意されています。

In [ ]:
%%sql
    
SELECT committed_at,snapshot_id,operation  FROM demo.simple_table.snapshots ORDER BY committed_at;

この情報を元に、以下のようにスナップショット ID を指定して、過去のデータを参照することができます。  

In [ ]:
%%sql

-- INSERT 後、UPDATE 前のテーブルにクエリ
SELECT * FROM demo.simple_table VERSION AS OF <INSERT 後、UPDATE 前のスナップショットIDを指定>;